In [ ]:
! pip install spacy clean-text unidecode

In [ ]:
! python -m spacy download de_core_news_lg

In [2]:
from collections import Counter
from pathlib import Path

import spacy
import cleantext

In [3]:
nlp = spacy.load('de_core_news_lg')

In [4]:
input_txt = Path('nsu.txt').read_text()
input_txt = cleantext.clean(input_txt, lang='de', lower=False)

In [8]:
nlp.max_length = len(input_txt)
doc = nlp(input_txt)
sentences = [[t.string for t in sent] for sent in doc.sents]

In [9]:
len(sentences)

13007

In [10]:
# only select sentences with over 5 words
cleaned_sents = list(filter(lambda x: len(x) > 5, sentences))

In [11]:
len(cleaned_sents)

9836

In [12]:
# count sentences
first_counter = Counter([''.join(x) for x in cleaned_sents])

In [14]:
# count second time but don't consider space / whitespaces
input_txt_minimal = cleantext.clean(input_txt, lang='de', no_punct=True, no_line_breaks=True)
second_counter = Counter()

In [15]:
for x in first_counter.most_common(50):
    cleaned_x = cleantext.clean(x[0], lang='de', no_line_breaks=True, no_punct=True)
    # only consider proper sentences
    if not x[0].strip().endswith('.'):
        continue
    final_count = input_txt_minimal.count(cleaned_x)
    second_counter[x[0].strip()] = final_count

In [16]:
second_counter.most_common(30)

[('Sie sagte eine sorgfältige Beobachtung der Umgebung ihrer gemeinsamen Wohnung und Zentrale der Vereinigung zu sowie eine schnelle und umsichtige Reaktion auf Vorkommnisse, die den Eindruck des unauffälligen bürgerlichen Lebens der drei Personen in Frage stellen könnten.',
  25),
 ('Die Angeklagte Zschäpe war bei ihren Handlungen in vollem Umfang schuldfähig.',
  25),
 ('In der öffentlichen Wahrnehmung sollte nur die Vereinigung NSU und deren Wirken präsent sein.',
  20),
 ('Die Öffentlichkeit sollte vom Fortbestand einer handlungsfähigen, schlagkräftigen, im Einzelnen nicht greifbaren Vereinigung ausgehen.',
  20),
 ('Keinesfalls wollten sie, dass die Ermittlungsbehörden und damit die Öffentlichkeit Einblicke in die Struktur der Organisation, die Anzahl und die Identität ihrer Mitglieder, deren Leben und Zusammenwirken sowie die Identität ihrer Unterstützer und die Art der Unterstützungshandlungen gewinnen würden.',
  20),
 ('Dadurch gab sie Uwe Böhnhardt und Uwe Mundlos die Sicherh

In [34]:
md_output = """| Rank | Anzahl | Satz |
| --- | --- | :--- |
"""
prev_n = None
rank = None
for rank, (value, n) in enumerate(second_counter.most_common(30)):
    if prev_n and prev_n == n:
        rank = prev_rank
    prev_rank = rank
    prev_n = n
    print(rank, n, value)
    md_output += f"| {rank + 1}. | {n} | {value} |\n"

0 25 Sie sagte eine sorgfältige Beobachtung der Umgebung ihrer gemeinsamen Wohnung und Zentrale der Vereinigung zu sowie eine schnelle und umsichtige Reaktion auf Vorkommnisse, die den Eindruck des unauffälligen bürgerlichen Lebens der drei Personen in Frage stellen könnten.
0 25 Die Angeklagte Zschäpe war bei ihren Handlungen in vollem Umfang schuldfähig.
2 20 In der öffentlichen Wahrnehmung sollte nur die Vereinigung NSU und deren Wirken präsent sein.
2 20 Die Öffentlichkeit sollte vom Fortbestand einer handlungsfähigen, schlagkräftigen, im Einzelnen nicht greifbaren Vereinigung ausgehen.
2 20 Keinesfalls wollten sie, dass die Ermittlungsbehörden und damit die Öffentlichkeit Einblicke in die Struktur der Organisation, die Anzahl und die Identität ihrer Mitglieder, deren Leben und Zusammenwirken sowie die Identität ihrer Unterstützer und die Art der Unterstützungshandlungen gewinnen würden.
2 20 Dadurch gab sie Uwe Böhnhardt und Uwe Mundlos die Sicherheit, ungefährdet in die Zentrale 

In [35]:
Path('table.md').write_text(md_output)

7008